# RAG with Gemini Pro

Note: Place your Google API key in the Google API Key folder of the directory this code is in, as the contents of a file called 'data-engineering-project.txt'.

In [1]:
# Packages
from RAG_Functions import *
import time
from sentence_transformers import SentenceTransformer
from pymilvus import Collection, connections

# Google API Setup
import google.generativeai as genai
import os
# Load API key from './Google API Key/data-engineering-project.txt'
with open(os.path.expanduser('./Google API Key/data-engineering-project.txt')) as f:
    GOOGLE_API_KEY = f.read().strip()
genai.configure(api_key=GOOGLE_API_KEY)

c:\Users\ijyli\miniforge3\envs\data_engineering_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Embedding Model

In [2]:
# embedding model
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

## Chat Model

In [3]:
chat_model = genai.GenerativeModel('gemini-1.0-pro-latest')
chat_model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

## Milvus Connection

In [4]:
connections.connect(host='localhost', port='19530')
collection = Collection("text_embeddings")      # Get an existing collection.
collection.load()

## Perform Chat

In [5]:
# Chat with model
input_text = input()

# Get embedding of input
input_embedding = get_mixedbread_of_query(embedding_model, input_text)

# Start timing query
start_time = time.time()

# Top5 sentences
top5_sentences, documents_cited, milvus_query_time = return_top_5_sentences(collection, input_embedding)

# End timing query
end_time = time.time()

# query time
query_time = end_time - start_time

print(top5_sentences)

['AppleServices: In third-party apps, you may see ads based on other information.', 'AppleServices: Apple may use this information in our marketing and advertising services.', 'AppleServices: At times Apple may provide third parties with certain personal information to provide or improve our products and services, including to deliver products at your request, or to help Apple market to consumers.', "AppleServices: You agree that Apple has the right, without liability to you, to disclose any Registration Data and/or Account information to law enforcement authorities, government officials, and/or a third party, as Apple believes is reasonably necessary or appropriate to enforce and/or verify compliance with any part of this Agreement (including but not limited to Apple's right to cooperate with any legal process relating to your use of the Service and/or products, and/or a third-party claim that your use of the Service and/or products is unlawful and/or infringes such third party's righ

In [6]:
# Construct prompt
prompt_lines = ["Context That May Be Helpful (You May Disregard if Not Helpful):"] + top5_sentences + ["User Query:\n" + input_text]
prompt = "\n".join(prompt_lines)
print(prompt)

Context That May Be Helpful (You May Disregard if Not Helpful):
AppleServices: In third-party apps, you may see ads based on other information.
AppleServices: Apple may use this information in our marketing and advertising services.
AppleServices: At times Apple may provide third parties with certain personal information to provide or improve our products and services, including to deliver products at your request, or to help Apple market to consumers.
AppleServices: You agree that Apple has the right, without liability to you, to disclose any Registration Data and/or Account information to law enforcement authorities, government officials, and/or a third party, as Apple believes is reasonably necessary or appropriate to enforce and/or verify compliance with any part of this Agreement (including but not limited to Apple's right to cooperate with any legal process relating to your use of the Service and/or products, and/or a third-party claim that your use of the Service and/or products

In [7]:
# Get response
response = chat_model.generate_content(prompt)
# Format response for user
response_for_user = response.text + "\nDocuments Cited: " + ', '.join(documents_cited) + "\nMilvus Query Time: " + str(round(milvus_query_time, 2)) + ' seconds'
print(response_for_user)

Information in the context does not explicitly state whether Apple sells users' information to third-party advertisers. However, it does state that Apple may share users' information with third parties for marketing and advertising purposes
Documents Cited: AppleServices_GameCenterTermsandCondi.txt, AppleServices_iTunesTermsofService.txt, AppleServices_PrivacyPolicy.txt
Milvus Query Time: 0.35 seconds
